# Décorateurs

## Motivations

Un décorateur permet de modifier le comportement d'une fonction, sans devoir changer le code de la fonction.

On peut appliquer un même décorateur à plusieurs fonctions différentes.

Avantages :
- éviter de la duplication
- séparer les préoccupations
- brique réutilisable

## Un décorateur très minimal

Un décorateur est une fonction qui renvoie une fonction.

In [38]:
def nop(func):
    return func

@nop
def fonction():
    print("bonjour")
    
fonction()

bonjour


La syntaxe `@` est juste une commodité :

In [39]:
def fonction():
    print("bonjour")

fonction = nop(fonction)  # équivalent

fonction()

bonjour


## Un décorateur qui fait quelque chose

La plupart du temps, un décorateur va suivre ce modèle, qui consiste à « emballer » l'appel à la fonction décorée :

In [ ]:
def decorateur_type(func):
    def wrapper(*args, **kwargs):
        # ici on peut insérer des choses à faire avant l'appel
        res = func(*args, **kwargs)
        # ici on peut insérer des choses à faire après l'appel
        return res
    return wrapper

Par exemple, on peut écrire ainsi un décorateur qui va mesurer le temps d'exécution de la fonction à chaque appel :

In [1]:
from time import monotonic

def chrono(func):
    def wrapper(*args, **kwargs):
        départ = monotonic()  # avant
        res = func(*args, **kwargs)
        print("Durée : {:.1f}ms".format((monotonic() - départ) * 1000))  # après
        return res
    return wrapper

In [2]:
@chrono
def fonction_courte():
    print("bonjour")

fonction_courte()

bonjour
Durée : 0.0ms


In [3]:
from time import sleep

@chrono
def fonction_longue():
    sleep(0.1)
    print("bonjour")

fonction_longue()

bonjour
Durée : 93.0ms


## Décorateur paramétré

Ça se complique un peu : on écrit une fonction qui va renvoyer un décorateur...

In [6]:
def retry(max_tries):
    def decorator(func):
        def wrapper(*args, **kwargs):
            tries = 1
            while True:
                try:
                    return func(*args, **kwargs)
                except Exception:
                    if tries >= max_tries:
                        raise
                tries += 1
        return wrapper
    return decorator

@retry(3)
def fonction():
    print("bonjour")
    1 / 0

fonction()

bonjour
bonjour
bonjour


ZeroDivisionError: division by zero

## Décorer son décorateur : @wraps

In [7]:
def decorateur(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@decorateur
def fonction():
    """Ma fonction"""
    print("bonjour")

In [8]:
print(fonction.__name__)

wrapper


In [19]:
print(fonction.__doc__)

None


In [9]:
from functools import wraps

def decorateur(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@decorateur
def fonction():
    """Ma fonction"""
    print("bonjour")

In [10]:
print(fonction.__name__)

fonction


In [22]:
print(fonction.__doc__)

Ma fonction


## Décorateurs multiples

Lorsqu'il y a plusieurs décorateurs, ils sont appliqués de bas en haut :

In [13]:
def decorateur1(func):
    def wrapper(*args, **kwargs):
        print(" avant 1")
        res = func(*args, **kwargs)
        print(" après 1")
        return res
    return wrapper

def decorateur2(func):
    def wrapper(*args, **kwargs):
        print("avant 2")
        res = func(*args, **kwargs)
        print("après 2")
        return res
    return wrapper

@decorateur2
@decorateur1
def fonction():
    print("  bonjour")

# équivalent à `fonction = decorateur2(decorateur1(fonction))`

Cela veut dire qu'à l'exécution on commencera par le plus extérieur, donc de haut en bas :

In [12]:
fonction()

avant 2
 avant 1
  bonjour
 après 1
après 2


## Alternative : écrire un décorateur avec une classe

In [14]:
from functools import update_wrapper

class compteur:
    def __init__(self, func):
        self.func = func
        update_wrapper(self, func)  # équivalent au @wraps dans le cas d'une fonction
        self.appels = 0

    def __call__(self, *args, **kwargs):
        self.appels += 1
        res = self.func(*args, **kwargs)
        print(self.appels, "appel(s)")
        return res

@compteur
def fonction():
    print("bonjour")

In [15]:
print(fonction.__name__)

fonction


In [20]:
fonction()

bonjour
5 appel(s)


In [45]:
fonction()

bonjour
2 appel(s)


## Context managers

In [ ]:
f = open("custom.css")
try:
    f.read()
finally:
    f.close()

In [21]:
with open("custom.css") as f:
    f.read()

## Décorateur et *context manager* à la fois

Il est possible de créer une classe qui fonctionne à la fois comme décorateur et comme *context manager*.

In [22]:
from contextlib import ContextDecorator

class contexte(ContextDecorator):
    def __enter__(self):
        print("Avant")
        return self

    def __exit__(self, *exc):
        print("Après")
        return False

Usage comme décorateur :

In [23]:
@contexte()
def fonction():
    print("Ma fonction")

fonction()

Avant
Ma fonction
Après


Usage comme *context manager* :

In [24]:
with contexte():
    print("Mon bloc")

Avant
Mon bloc
Après


In [25]:
with contexte():
    1 / 0

Avant
Après


ZeroDivisionError: division by zero

## Références

- https://www.python.org/dev/peps/pep-0318/
- [Practical decorators (PyCon 2019)](https://speakerdeck.com/pycon2019/reuven-m-lerner-practical-decorators)
- [Decorators unwrapped (PyCon 2017)](https://speakerdeck.com/pycon2017/katie-silverio-decorators-unwrapped-how-do-they-work)
- https://wrapt.readthedocs.io/
- https://docs.python.org/3.8/library/contextlib.html#contextlib.ContextDecorator